In [87]:
import requests
import json
import pandas as pd
import urllib
import json
import pandas as pd
import io
import requests

In [19]:
url="http://dati.ustat.miur.it/dataset/3dd9ca7f-9cc9-4a1a-915c-e569b181dbd5/resource/1fc15513-0eeb-48c7-b541-2b38d86ea9ea/download/iscrittixresidenzasedecorsoateneo.csv"
#s=requests.get(url).content
c=pd.read_csv(url, sep=';', encoding='latin-1')


In [3]:
c

,AnnoA,AteneoCOD,AteneoNOME,SedeP,ResidenzaR,IstatP,ResidenzaP,Isc
0,2019/2020,101,Torino,1,ABRUZZO,66,L'AQUILA,76
1,2019/2020,101,Torino,1,ABRUZZO,67,TERAMO,70
2,2019/2020,101,Torino,1,ABRUZZO,68,PESCARA,82
3,2019/2020,101,Torino,1,ABRUZZO,69,CHIETI,114
4,2019/2020,101,Torino,1,BASILICATA,76,POTENZA,217
...,...,...,...,...,...,...,...,...
69153,2019/2020,9201,Cagliari,dati aggregati,SICILIA,dati aggregati,dati aggregati,17
69154,2019/2020,9201,Cagliari,dati aggregati,TOSCANA,dati aggregati,dati aggregati,16
69155,2019/2020,9201,Cagliari,dati aggregati,UMBRIA,dati aggregati,dati aggregati,7
69156,2019/2020,9201,Cagliari,dati aggregati,VALLE D'AOSTA,dati aggregati,dati aggregati,1


In [88]:
students_dictionary = {'AnnoA': [], 'AteneoNOME': [], 'ResidenzaR': [],'ResidenzaP': []}

In [89]:
def getFileVecchio():
    url = 'http://dati.ustat.miur.it/dataset/3dd9ca7f-9cc9-4a1a-915c-e569b181dbd5/resource/1fc15513-0eeb-48c7-b541-2b38d86ea9ea/download/iscrittixresidenzasedecorsoateneo.csv'
    urllib.request.urlretrieve(url, '/home/alesssandro/Scrivania/students.csv')
    saved = pd.read_csv("/home/alesssandro/Scrivania/students.csv", sep = ';', encoding = "latin-1")
    keep_col = ['AnnoA', 'AteneoNOME', 'ResidenzaR', 'ResidenzaP']  # removing unnecessary columns
    new_file = saved[keep_col]
    new_file.to_csv("/home/alesssandro/Scrivania/students.csv", index=False, sep = ';')


In [90]:
def getFile():
    url = 'http://dati.ustat.miur.it/dataset/3dd9ca7f-9cc9-4a1a-915c-e569b181dbd5/resource/1fc15513-0eeb-48c7-b541-2b38d86ea9ea/download/iscrittixresidenzasedecorsoateneo.csv'
    urllib.request.urlretrieve(url, '/home/alesssandro/Scrivania/students.csv')
    saved = pd.read_csv("/home/alesssandro/Scrivania/students.csv", sep = ';', encoding = "latin-1")
    data = saved.drop(['Isc', 'AteneoCOD', 'IstatP', 'SedeP'], axis = 1, inplace = True)#removing unnecessary columns
    saved.to_csv("/home/alesssandro/Scrivania/students.csv", sep = ';', encoding = "latin-1", index = None)


In [91]:
def queryData():
    query = """SELECT * from "1fc15513-0eeb-48c7-b541-2b38d86ea9ea" WHERE "AnnoA" > '2018/2019'"""
    url2 = 'http://dati.ustat.miur.it/api/3/action/datastore_search_sql?sql={}'.format(query.replace(" ", ""))
    fileobj = urllib.request.urlopen(url2)
    decoded = fileobj.read().decode()
    json_data = json.loads(decoded)
    results = pd.DataFrame(json_data['result']['records'])
    return results

In [92]:
def getLastRow():
    df1 = pd.read_csv('/home/alesssandro/Scrivania/students.csv', sep = ';', encoding = "latin-1")
    df2 = pd.DataFrame(df1.iloc[-1:,:].values) #get last row
    last_row = df2._get_value(0,0)#.replace("-", "") # date of last row
    return last_row
    

In [97]:
def get_values(key, date):
    query = """SELECT * from "1fc15513-0eeb-48c7-b541-2b38d86ea9ea" WHERE "AnnoA" > {}""".format(date)
    url2 = 'http://dati.ustat.miur.it/api/3/action/datastore_search_sql?sql={}'.format(query.replace(" ", ""))
    fileobj = urllib.request.urlopen(url2)
    decoded = fileobj.read().decode()
    json_data = json.loads(decoded)
    results = pd.DataFrame(json_data['result']['records'])
    
    for row in range (len(results)):
        students_dictionary[key].append(results[key][row])

In [94]:
def intoCsv():
    df2 = pd.DataFrame(data = students_dictionary)
    df2.to_csv('/home/alesssandro/Scrivania/students.csv', mode='a', index=None, header=False)

In [98]:
def addNewRow():
    getLastRow()
    get_values('AnnoA', last_row)
    get_values('AteneoNOME', last_row)
    get_values('ResidenzaR', last_row)
    get_values('ResidenzaP', last_row)
    intoCsv()
    

In [99]:
try:
    file = open("/home/alesssandro/Scrivania/students.csv")
    addNewRow()
    
except IOError:
    getFile()

NameError: name 'last_row' is not defined

In [54]:
pd.read_csv('/home/alesssandro/Scrivania/students.csv', sep=';', encoding = 'latin-1')

,AnnoA,AteneoNOME,ResidenzaR,ResidenzaP
0,2019/2020,Torino,ABRUZZO,L'AQUILA
1,2019/2020,Torino,ABRUZZO,TERAMO
2,2019/2020,Torino,ABRUZZO,PESCARA
3,2019/2020,Torino,ABRUZZO,CHIETI
4,2019/2020,Torino,BASILICATA,POTENZA
...,...,...,...,...
69153,2019/2020,Cagliari,SICILIA,dati aggregati
69154,2019/2020,Cagliari,TOSCANA,dati aggregati
69155,2019/2020,Cagliari,UMBRIA,dati aggregati
69156,2019/2020,Cagliari,VALLE D'AOSTA,dati aggregati


In [8]:
response = requests.post(url, data=my_data, headers=my_headers)

responseJson = response.json()
rows = []
for item in responseJson['data']:
    
        rows.append(csvRow)
keywords = pd.DataFrame(rows, columns=['Year', 'Ateneo', 'Residenza'])
students.to_csv('./students_report.csv', index=None)

NameError: name 'url' is not defined